Given the output we get from the "basic_neural_network" notebook, the accuracy we get when testing our neural network on new data is poor even when we train it with large data set (24000 sequences). <br/>
This is of course related to the fact that we only make use of (part of) the DNA sequence itself and no extra information is incorporated in the input. <br/>
This notebook trys to incorporate the Transcription Factor Binding Site information into the current data set and  explores different ways to accomplish this.

### To Do
1. Merge the information of TF binding sites in various ways and prepare it properly into the desired format.
2. Design the Neural Network architecture (without implementation)
3. Hopefully, test some architecture designs and see how it improves(?) the result accuracy

### Some Design Ideas

- **Example Sequence: ACGT with TFBS score 0.8 for TF1 and score 0.6 for TF2**

Currently, this example sequence is translated into \[1, 0, 0, 0, || 0, 1, 0, 0, || 0, 0, 1, 0, || 0, 0, 0, 1\] without incorporation of the TFBS information.
- **Final Training Data Input**: <br/>Reading all sequences aligned with **2000 bases** from **3000 files** would give us a matrix (all entries of *value 0 or 1*) in the shape of: $\mathbf{72000\times8000}$, which would be final the training data set that is fed into the neural network.

#### Idea 1:  Naive Approach: Append At The End
Simply Append the score and the end of each and every base of a particular sequence.
The example sequence would be translated into: <br/> \[1, 0, 0, 0, 0.8, 0.6, || 0, 1, 0, 0, 0.8, 0.6, || 0, 0, 1, 0, 0.8, 0.6, || 0, 0, 0, 1, 0.8, 0.6\]
- **Final Training Data Input:** <br/> Suppose there are **20** Transcription Factors in total. $(20+4)\times2000=48000$ 
<br/> Matrix (entries of *floating values*) in the shape of: $\mathbf{72000\times48000}$
- **Problems:** <br/> This approach treats 1, 0 and the TFBS score as "equally important". The base (AGCT) info is "weighting" (contributing) four values each in the final training data whereas the TFBS info is contributing one value for one TF, which may be problematic. 

#### Idea 2: Train Two Neural Networks Separately
We can train two neural networks separately and combine the results at the last (or the last n th) step. <br/>
The first neural network (NN1) is just like what we did previously, simply ignore the TFBS informaiton. The second neural network (NN2) uses only TFBS values, also uses sequential neural network to predict outcomes. At the last step (or the last n th step), we combine the two neural networks together, basically by merging the neurons from NN1 and NN2 together.
- **Final Training Data Input:** <br/> 
For **NN1**: Matrix (entries of *value 0, 1*) in the shape of: $72000\times8000$, Output Matrix in the shape of $72000\times25$ <br/>
For **NN2**: Matrix (entries of *floating values*) in the shape of: $72000\times40000$, Output Matrix in the shape of $72000\times35$ <br/>
For **NN3**: Matrix (entries of *floating values*) in the shape of: $72000\times60$, Output Matrix in the shape of $72000\times1$ <br/>
*Example*: For each sequence, NN1 transforms 8000 values to 25 values(neurons), NN2 transforms 40000 values to 35 values(neurons). Then, we combine these 60 values together, use a new neural network (NN3) to make the final prediction based on these 60 values. <br/>
*Note*: 25 and 35 are in the above example are just values that are arbitrarily chosen and set, they really can be any other value that may improve the performance of the entire neural network.
- **Comments:** <br/> This approach makes it possible to manually adjust the weights between the two parts of info (base info and TFBS info) and tune it into any balance in order to get better performance. <br/> However, it seems to lose the "corresponding relationship" between base info and TFBS info. It may be difficult for a neural network to precisely link the base info and TFBS info for each sequence all by itself as they are not tied closely in the raw inpput.

#### Idea 3: Mixed Information: Multiply and Flatten
Instead of treaing the base info and the TFBS info as two parts and trying to "combine" them as in previous two approaches, we "mix" them. "Mix" can be carried out by multiplication for each and every transcription factor. <br/>
For example, the example sequence can become: <br/>
\[ 0.8, 0, 0, 0, 0.6, 0, 0, 0, || 0, 0.8, 0, 0, 0, 0.6, 0, 0, || 0, 0, 0.8, 0, 0, 0, 0.6, 0, || 0, 0, 0, 0.8, 0, 0, 0, 0.6 \]
- **Final Training Data Input:** <br/> Suppose there are **20** Transcription Factors in total. $20\times4\times2000=160000$ 
<br/> Matrix (entries of *floating values*) in the shape of: $\mathbf{72000\times160000}$
- **Comments:** <br/>
There will be too many parameters to trains, even when compared to the large amount of sequences we own. There will be more redundancy in the training data since the AGCT info is copied 20 times.

### Incorporate the TF Information and Prepare the Data

In [ ]:
base_pairs = {'A': [1, 0, 0, 0], 
'C': [0, 1, 0, 0],
'G': [0, 0, 1, 0],
'T': [0, 0, 0, 1],
'a': [1, 0, 0, 0],
'c': [0, 1, 0, 0],
'g': [0, 0, 1, 0],
't': [0, 0, 0, 1],
'n': [0, 0, 0, 0],
'N': [0, 0, 0, 0]}
tfbs  = {[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]}